In [1]:
import theano

class AXPBOp(theano.Op):
    """
    This creates an Op that takes x to a*x+b.
    """
    __props__ = ("a", "b")

    def __init__(self, a, b):
        self.a = a
        self.b = b
        super(AXPBOp, self).__init__()

    def make_node(self, x):
        # check that the theano version has support for __props__.
        assert hasattr(self, '_props'), ("Your version of theano is too"
                                         "old to support __props__.")
        x = theano.tensor.as_tensor_variable(x)
        return theano.Apply(self, [x], [x.type()])

    def perform(self, node, inputs, output_storage):
        x = inputs[0]
        z = output_storage[0]
        z[0] = self.a * x + self.b

    def infer_shape(self, node, i0_shapes):
        return i0_shapes

    def grad(self, inputs, output_grads):
        return [a * output_grads[0] + b]

In [2]:
import theano.tensor as T

a = T.dvector(name="a")
b = T.dvector(name="b")
c = T.sum(T.log(a + b))
c.name="c"

op = theano.OpFromGraph([a], [c])

{'b8736340980489': 8736288375301, 'Elemwise{add,no_inplace}(a, b)8736288412021': 8736288373245, 'Elemwise{log,no_inplace}.08736288412085': 8736288375309, 'Elemwise{add,no_inplace}.08736288412037': 8736288375305, 'Sum{acc_dtype=float64}(Elemwise{log,no_inplace}.0)8736288373225': 8736288373237, 'a8736340980733': 8736288373233, 'c8736288412097': 8736288375313, 'Elemwise{log,no_inplace}(Elemwise{add,no_inplace}.0)8736288412069': 8736288373241}


In [3]:
d = T.dvector(name="d")
result = op(d)
result.name="results"

In [4]:
print map(hash, [a,b,c,d])

[8736340980733, 8736340980489, 8736288412097, 8736340977325]


In [5]:
f = theano.function([b, d], result, on_unused_input="warn")

{'node': <theano.compile.builders.OpFromGraph object at 0x7f2135e11a90>(d), 'f': (<frame object at 0x2153130>, '/home/stephan/GitProjects/Theano/theano/compile/builders.py', 130, 'make_thunk', ['        for f in inspect.stack():\n'], 0), 'self': <theano.compile.builders.OpFromGraph object at 0x7f2135e11a90>, 'storage_map': {b: [None], d: [None], results: [None]}, 'no_recycling': set([results]), 'compute_map': {b: [True], d: [True], results: [False]}}

{'node': <theano.compile.builders.OpFromGraph object at 0x7f2135e11a90>(d), 'profiler': None, 'fgraph': [<theano.compile.builders.OpFromGraph object at 0x7f2135e11a90>(d)], 'thunks': [], 'k': results, 'order': [<theano.compile.builders.OpFromGraph object at 0x7f2135e11a90>(d)], 'self': <theano.gof.vm.VM_Linker object at 0x7f2168026550>, 'dependencies': {b: [], d: [], results: []}, 'storage_map': {b: [None], d: [None], results: [None]}, 'var': d, 'compute_map_re': defaultdict(<function <lambda> at 0x7f2135ce8e60>, {b: [1], d: [1], results:

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 0 is not part of the computational graph needed to compute the outputs: b.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.
  if __name__ == '__main__':


MissingInputError: ('The following error happened while compiling the node', <theano.compile.builders.OpFromGraph object at 0x7f2135e11a90>(d), '\n', 'An input of the graph, used to compute Elemwise{add,no_inplace}(a, b), was not provided and not given a value.Use the Theano flag exception_verbosity=\'high\',for more information on this error.\n\nBacktrace when the variable is created:\n  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 260, in dispatcher\n    return self.dispatch_shell(stream, msg)\n  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 212, in dispatch_shell\n    handler(stream, idents, msg)\n  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelbase.py", line 370, in execute_request\n    user_expressions, allow_stdin)\n  File "/usr/local/lib/python2.7/dist-packages/ipykernel/ipkernel.py", line 175, in do_execute\n    shell.run_cell(code, store_history=store_history, silent=silent)\n  File "/usr/local/lib/pyt

In [ ]:
%debug

In [ ]:
f([5,5,5], [1,2,3])

In [ ]:
theano.function?